# **HR Analytics Data Wrangling & Modeling Preparation**

---



This project addresses a common challenge in HR Analytics by integrating multiple disparate data tables—Employee Master, Performance Ratings, and Lookup Tables—into a single, clean, and unified dataset. Leveraging SQLite queries within Python, the process successfully joins relational data, engineers a new Tenure feature (Cell 74), standardizes categorical variables using One-Hot Encoding (Cell 67), and handles missing performance scores via mode imputation (Cell 70). The final output delivers two distinct, saved datasets: one for Descriptive Reporting and one optimized for subsequent Machine Learning Modeling, ensuring the data is structured, complete, and ready for advanced analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# **Data Ingestion and Inspection**

In [ ]:

# --- 1. Main Employee Data (Fact Table) ---
df_employee = pd.read_csv('Employee.csv')

# --- 2. Lookup/Dimension Tables ---

# Lookup for Education
df_education = pd.read_csv('EducationLevel.csv')

# Lookup for Performance Rating
df_performance_rating = pd.read_csv('PerformanceRating.csv')

# Lookup for Rating Levels
df_rating_level = pd.read_csv('RatingLevel.csv')

# Lookup for Satisfaction Levels (Used to decode satisfaction IDs into text)
df_satisfied_level = pd.read_csv('SatisfiedLevel.csv')


In [ ]:
display(df_employee.head())

,EmployeeID,FirstName,LastName,Gender,Age,BusinessTravel,Department,DistanceFromHome (KM),State,Ethnicity,...,MaritalStatus,Salary,StockOptionLevel,OverTime,HireDate,Attrition,YearsAtCompany,YearsInMostRecentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,3012-1A41,Leonelle,Simco,Female,30,Some Travel,Sales,27,IL,White,...,Divorced,102059,1,No,2012-01-03,No,10,4,9,7
1,CBCB-9C9D,Leonerd,Aland,Male,38,Some Travel,Sales,23,CA,White,...,Single,157718,0,Yes,2012-01-04,No,10,6,10,0
2,95D7-1CE9,Ahmed,Sykes,Male,43,Some Travel,Human Resources,29,CA,Asian or Asian American,...,Married,309964,1,No,2012-01-04,No,10,6,10,8
3,47A0-559B,Ermentrude,Berrie,Non-Binary,39,Some Travel,Technology,12,IL,White,...,Married,293132,0,No,2012-01-05,No,10,10,10,0
4,42CC-040A,Stace,Savege,Female,29,Some Travel,Human Resources,29,CA,White,...,Single,49606,0,No,2012-01-05,Yes,6,1,1,6


In [ ]:
print("Employee Columns:")
print(df_employee.columns.tolist())

Employee Columns:
['EmployeeID', 'FirstName', 'LastName', 'Gender', 'Age', 'BusinessTravel', 'Department', 'DistanceFromHome (KM)', 'State', 'Ethnicity', 'Education', 'EducationField', 'JobRole', 'MaritalStatus', 'Salary', 'StockOptionLevel', 'OverTime', 'HireDate', 'Attrition', 'YearsAtCompany', 'YearsInMostRecentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


In [ ]:
display(df_education.head())

,EducationLevelID,EducationLevel
0,1,No Formal Qualifications
1,2,High School
2,3,Bachelors
3,4,Masters
4,5,Doctorate


In [ ]:
display(df_performance_rating.head())

,PerformanceID,EmployeeID,ReviewDate,EnvironmentSatisfaction,JobSatisfaction,RelationshipSatisfaction,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,WorkLifeBalance,SelfRating,ManagerRating
0,PR01,79F7-78EC,1/2/2013,5,4,5,1,0,4,4,4
1,PR02,B61E-0F26,1/3/2013,5,4,4,1,3,4,4,3
2,PR03,F5E3-48BB,1/3/2013,3,4,5,3,2,3,5,4
3,PR04,0678-748A,1/4/2013,5,3,2,2,0,2,3,2
4,PR05,541F-3E19,1/4/2013,5,2,3,1,0,4,4,3


In [ ]:
display(df_rating_level.head())

,RatingID,RatingLevel
0,1,Unacceptable
1,2,Needs Improvement
2,3,Meets Expectation
3,4,Exceeds Expectation
4,5,Above and Beyond


In [ ]:
display(df_satisfied_level.head())

,SatisfactionID,SatisfactionLevel
0,1,Very Dissatisfied
1,2,Dissatisfied
2,3,Neutral
3,4,Satisfied
4,5,Very Satisfied


# **Data Integration via SQLite Query**

In [ ]:
import sqlite3
import pandas as pd

# 1. Create an in-memory SQLite database connection
conn = sqlite3.connect(':memory:')

# 2. Write DataFrames to SQL tables using their variable names
df_employee.to_sql('df_employee', conn, index=False, if_exists='replace')
df_education.to_sql('df_education', conn, index=False, if_exists='replace')
df_satisfied_level.to_sql('df_satisfied_level', conn, index=False, if_exists='replace')
df_performance_rating.to_sql('df_performance_rating', conn, index=False, if_exists='replace')
df_performance_rating.to_sql('df_performance_rating', conn, index=False, if_exists='replace')
df_rating_level.to_sql('df_rating_level', conn, index=False, if_exists='replace')

print("DataFrames successfully loaded into the SQLite database.")

DataFrames successfully loaded into the SQLite database.


# **Initial Review and Column Ordering**

In [ ]:
sql_full_combined_query = """
SELECT
    -- Employee Details (from df_employee)
    e.EmployeeID,
    e.Gender,
    e.Age,
    e.BusinessTravel,
    e.Department,
    e."DistanceFromHome (KM)",
    e.State,
    e.Ethnicity,
    e.JobRole,
    e.MaritalStatus,
    e.Salary,
    e.StockOptionLevel,
    e.OverTime,
    e.HireDate,
    e.Attrition,
    e.YearsAtCompany,
    e.YearsInMostRecentRole,
    e.YearsSinceLastPromotion,
    e.YearsWithCurrManager,

    -- Education Text (from df_education)
    el.EducationLevel,
    e.EducationField,
    e.Education,

    -- Performance Metrics (from df_performance_rating)
    pr.ReviewDate,
    pr.EnvironmentSatisfaction,
    pr.JobSatisfaction,
    pr.WorkLifeBalance,
    pr.RelationshipSatisfaction,
    pr.TrainingOpportunitiesWithinYear,
    pr.TrainingOpportunitiesTaken,
    pr.SelfRating,
    pr.ManagerRating,

    -- Satisfaction Text Joins (from df_satisfied_level)
    sl.SatisfactionLevel AS JobSatisfaction_Text,
    esl.SatisfactionLevel AS EnvironmentSatisfaction_Text,

    -- Rating Text (from df_rating_level, joined twice)
    sr.RatingLevel AS SelfRating_Text,
    mr.RatingLevel AS ManagerRating_Text

FROM
    df_employee e

-- 1. Link Education Text
LEFT JOIN
    df_education el ON e.Education = el.EducationLevelID

-- 2. Link Performance Metrics
LEFT JOIN
    df_performance_rating pr ON e.EmployeeID = pr.EmployeeID

-- 3. Link Job Satisfaction Text (Links to PR table's JobSatisfaction)
LEFT JOIN
    df_satisfied_level sl ON pr.JobSatisfaction = sl.SatisfactionID

-- 4. Link Environment Satisfaction Text (***NEW JOIN***)
LEFT JOIN
    df_satisfied_level esl ON pr.EnvironmentSatisfaction = esl.SatisfactionID

-- 5. Link Self Rating Text
LEFT JOIN
    df_rating_level sr ON pr.SelfRating = sr.RatingID

-- 6. Link Manager Rating Text
LEFT JOIN
    df_rating_level mr ON pr.ManagerRating = mr.RatingID;
"""

# Execute the query and load the result back into a single pandas DataFrame
df_combined_data = pd.read_sql_query(sql_full_combined_query, conn)

print("--- Fully Combined HR Analytics Dataset ---")
# Display a sample of the new columns
display(df_combined_data[[
    'EmployeeID',
    'YearsAtCompany',
    'BusinessTravel',
    'JobSatisfaction_Text',
    'EnvironmentSatisfaction_Text'
]].head())

--- Fully Combined HR Analytics Dataset ---


,EmployeeID,YearsAtCompany,BusinessTravel,JobSatisfaction_Text,EnvironmentSatisfaction_Text
0,3012-1A41,10,Some Travel,Dissatisfied,Neutral
1,3012-1A41,10,Some Travel,Very Satisfied,Satisfied
2,3012-1A41,10,Some Travel,Dissatisfied,Satisfied
3,3012-1A41,10,Some Travel,Neutral,Neutral
4,3012-1A41,10,Some Travel,Satisfied,Satisfied


In [ ]:
# Check the number of unique employees in the combined data
print(f"Total Unique Employees in Combined Data: {df_combined_data['EmployeeID'].nunique()}")

# Display the first few rows including the ReviewDate
print("\n--- Combined HR Analytics Dataset (with ReviewDate) ---")
display(df_combined_data[['EmployeeID', 'ReviewDate', 'JobSatisfaction_Text', 'SelfRating_Text']].head(10))

Total Unique Employees in Combined Data: 1470

--- Combined HR Analytics Dataset (with ReviewDate) ---


,EmployeeID,ReviewDate,JobSatisfaction_Text,SelfRating_Text
0,3012-1A41,10/29/2020,Dissatisfied,Above and Beyond
1,3012-1A41,10/29/2021,Very Satisfied,Meets Expectation
2,3012-1A41,10/29/2022,Dissatisfied,Above and Beyond
3,3012-1A41,10/30/2016,Neutral,Meets Expectation
4,3012-1A41,10/30/2017,Satisfied,Meets Expectation
5,3012-1A41,10/30/2018,Very Satisfied,Above and Beyond
6,3012-1A41,10/30/2019,Neutral,Above and Beyond
7,3012-1A41,10/31/2014,Satisfied,Exceeds Expectation
8,3012-1A41,10/31/2015,Very Satisfied,Exceeds Expectation
9,CBCB-9C9D,7/29/2020,Dissatisfied,Meets Expectation


In [ ]:

print(df_combined_data.columns.tolist())

['EmployeeID', 'Gender', 'Age', 'BusinessTravel', 'Department', 'DistanceFromHome (KM)', 'State', 'Ethnicity', 'JobRole', 'MaritalStatus', 'Salary', 'StockOptionLevel', 'OverTime', 'HireDate', 'Attrition', 'YearsAtCompany', 'YearsInMostRecentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'EducationLevel', 'EducationField', 'Education', 'ReviewDate', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'RelationshipSatisfaction', 'TrainingOpportunitiesWithinYear', 'TrainingOpportunitiesTaken', 'SelfRating', 'ManagerRating', 'JobSatisfaction_Text', 'EnvironmentSatisfaction_Text', 'SelfRating_Text', 'ManagerRating_Text']


In [ ]:
print(df_combined_data.shape)

(6899, 35)


In [ ]:
# Display all columns and their data types
print(df_combined_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6899 entries, 0 to 6898
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   EmployeeID                       6899 non-null   object 
 1   Gender                           6899 non-null   object 
 2   Age                              6899 non-null   int64  
 3   BusinessTravel                   6899 non-null   object 
 4   Department                       6899 non-null   object 
 5   DistanceFromHome (KM)            6899 non-null   int64  
 6   State                            6899 non-null   object 
 7   Ethnicity                        6899 non-null   object 
 8   JobRole                          6899 non-null   object 
 9   MaritalStatus                    6899 non-null   object 
 10  Salary                           6899 non-null   int64  
 11  StockOptionLevel                 6899 non-null   int64  
 12  OverTime            

In [ ]:
df_combined_data.head()

,EmployeeID,Gender,Age,BusinessTravel,Department,DistanceFromHome (KM),State,Ethnicity,JobRole,MaritalStatus,...,WorkLifeBalance,RelationshipSatisfaction,TrainingOpportunitiesWithinYear,TrainingOpportunitiesTaken,SelfRating,ManagerRating,JobSatisfaction_Text,EnvironmentSatisfaction_Text,SelfRating_Text,ManagerRating_Text
0,3012-1A41,Female,30,Some Travel,Sales,27,IL,White,Sales Executive,Divorced,...,3.0,5.0,1.0,0.0,5.0,4.0,Dissatisfied,Neutral,Above and Beyond,Exceeds Expectation
1,3012-1A41,Female,30,Some Travel,Sales,27,IL,White,Sales Executive,Divorced,...,4.0,4.0,1.0,0.0,3.0,3.0,Very Satisfied,Satisfied,Meets Expectation,Meets Expectation
2,3012-1A41,Female,30,Some Travel,Sales,27,IL,White,Sales Executive,Divorced,...,2.0,4.0,1.0,0.0,5.0,4.0,Dissatisfied,Satisfied,Above and Beyond,Exceeds Expectation
3,3012-1A41,Female,30,Some Travel,Sales,27,IL,White,Sales Executive,Divorced,...,4.0,2.0,3.0,0.0,3.0,3.0,Neutral,Neutral,Meets Expectation,Meets Expectation
4,3012-1A41,Female,30,Some Travel,Sales,27,IL,White,Sales Executive,Divorced,...,2.0,5.0,3.0,1.0,3.0,2.0,Satisfied,Satisfied,Meets Expectation,Needs Improvement


In [ ]:
# List all columns in the desired, logically grouped order
new_column_order = [
    # 1. IDENTIFICATION & DEMOGRAPHICS (8 columns)
    'EmployeeID', 'Gender', 'Age', 'Ethnicity',
    'Department', 'JobRole', 'MaritalStatus', 'BusinessTravel',

    # 2. TENURE & STATUS (8 columns)
    'HireDate', 'Attrition', 'YearsAtCompany', 'YearsInMostRecentRole',
    'YearsSinceLastPromotion', 'YearsWithCurrManager', 'OverTime', 'StockOptionLevel',

    # 3. COMPENSATION & LOCATION (3 columns)
    'Salary',
    'DistanceFromHome (KM)',
    'State',

    # 4. EDUCATION (3 columns - NOW INCLUDES NUMERICAL ID)
    'Education',              # The numerical ID (e.Education)
    'EducationLevel',         # The text description
    'EducationField',

    # 5. PERFORMANCE & RATINGS (9 columns - NOW INCLUDES NUMERICAL IDs)
    'ReviewDate',
    'SelfRating',
    'SelfRating_Text',
    'ManagerRating',
    'ManagerRating_Text',
    'TrainingOpportunitiesWithinYear',
    'TrainingOpportunitiesTaken',

    # 6. SATISFACTION & BALANCE (6 columns)
    'JobSatisfaction', 'JobSatisfaction_Text',
    'EnvironmentSatisfaction', 'EnvironmentSatisfaction_Text',
    'WorkLifeBalance', 'RelationshipSatisfaction',
]

df_combined_data = df_combined_data[new_column_order]

# Display the new column list to verify
print("New Column Order:")
print(df_combined_data.columns.tolist())

New Column Order:
['EmployeeID', 'Gender', 'Age', 'Ethnicity', 'Department', 'JobRole', 'MaritalStatus', 'BusinessTravel', 'HireDate', 'Attrition', 'YearsAtCompany', 'YearsInMostRecentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'OverTime', 'StockOptionLevel', 'Salary', 'DistanceFromHome (KM)', 'State', 'Education', 'EducationLevel', 'EducationField', 'ReviewDate', 'SelfRating', 'SelfRating_Text', 'ManagerRating', 'ManagerRating_Text', 'TrainingOpportunitiesWithinYear', 'TrainingOpportunitiesTaken', 'JobSatisfaction', 'JobSatisfaction_Text', 'EnvironmentSatisfaction', 'EnvironmentSatisfaction_Text', 'WorkLifeBalance', 'RelationshipSatisfaction']


# **Feature Transformation (Encoding & Datetime)**

In [ ]:
# Convert 'Attrition' ('Yes'/'No') to 1/0
df_combined_data['Attrition'] = df_combined_data['Attrition'].replace({'Yes': 1, 'No': 0})

# Convert 'OverTime' ('Yes'/'No') to 1/0
df_combined_data['OverTime'] = df_combined_data['OverTime'].replace({'Yes': 1, 'No': 0})

# Display the data types and head to confirm the change
print("--- Data Types After Conversion ---")
print(df_combined_data[['Attrition', 'OverTime']].dtypes)
print("\n--- Head to Confirm Values ---")
display(df_combined_data[['Attrition', 'OverTime']].head())

--- Data Types After Conversion ---
Attrition    int64
OverTime     int64
dtype: object

--- Head to Confirm Values ---


/tmp/ipython-input-3870682794.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined_data['Attrition'] = df_combined_data['Attrition'].replace({'Yes': 1, 'No': 0})
/tmp/ipython-input-3870682794.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined_data['OverTime'] = df_combined_data['OverTime'].replace({'Yes': 1, 'No': 0})


,Attrition,OverTime
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [ ]:
# Convert HireDate to datetime format
df_combined_data['HireDate'] = pd.to_datetime(df_combined_data['HireDate'])

# Convert ReviewDate to datetime format
df_combined_data['ReviewDate'] = pd.to_datetime(df_combined_data['ReviewDate'])

# Verify the changes
print("--- Data Types After Datetime Conversion ---")
print(df_combined_data[['HireDate', 'ReviewDate']].dtypes)

--- Data Types After Datetime Conversion ---
HireDate      datetime64[ns]
ReviewDate    datetime64[ns]
dtype: object


In [ ]:
# List of columns to be One-Hot Encoded
# Note: We exclude the text rating columns as we have/will add their numerical IDs.
categorical_cols = [
    'Gender', 'BusinessTravel', 'Department', 'State',
    'Ethnicity', 'JobRole', 'MaritalStatus', 'EducationField'
]

# Apply One-Hot Encoding
# drop_first=True removes one dummy variable per feature group (e.g., 'Gender_Female')
# to avoid perfect correlation, which is important for regression models.
df_combined_data = pd.get_dummies(df_combined_data,
                                   columns=categorical_cols,
                                   drop_first=True,
                                   dtype=int)

# Display the new shape and the new columns
print(f"New DataFrame shape: {df_combined_data.shape}")
print("\nFirst 10 Columns to Show New Encoded Features:")
print(df_combined_data.columns.tolist()[:10])

New DataFrame shape: (6899, 64)

First 10 Columns to Show New Encoded Features:
['EmployeeID', 'Age', 'HireDate', 'Attrition', 'YearsAtCompany', 'YearsInMostRecentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'OverTime', 'StockOptionLevel']


# **Missing Value Analysis and Imputation**


In [ ]:
# Displays the count of nulls for every column in the DataFrame
print(df_combined_data.isnull().sum())

EmployeeID                            0
Age                                   0
HireDate                              0
Attrition                             0
YearsAtCompany                        0
                                     ..
EducationField_Information Systems    0
EducationField_Marketing              0
EducationField_Marketing              0
EducationField_Other                  0
EducationField_Technical Degree       0
Length: 64, dtype: int64


In [ ]:
# List of the numerical score columns that have NaNs (float64 dtype)
numerical_scores_to_impute = [
    'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance',
    'RelationshipSatisfaction', 'TrainingOpportunitiesWithinYear',
    'TrainingOpportunitiesTaken', 'SelfRating', 'ManagerRating'
]

# 1. Impute Numerical Scores with the Mode and Convert Type
for col in numerical_scores_to_impute:
    # Calculate the mode (most frequent value)
    mode_value = df_combined_data[col].mode()[0]

    # FIX: Assign the result of fillna() directly back to the column
    df_combined_data[col] = df_combined_data[col].fillna(mode_value).astype('int64')


# 2. Impute Text and Date Columns
text_cols_to_impute = [
    'JobSatisfaction_Text', 'EnvironmentSatisfaction_Text',
    'SelfRating_Text', 'ManagerRating_Text'
]
# Fill missing text descriptions with a clear placeholder
df_combined_data[text_cols_to_impute] = df_combined_data[text_cols_to_impute].fillna('No Review Data')

# Fill missing ReviewDate with a placeholder date
df_combined_data['ReviewDate'] = df_combined_data['ReviewDate'].fillna('1900-01-01')

# Verify the changes
print("--- After Imputation (Warnings Fixed) ---")
print(df_combined_data[['ReviewDate'] + numerical_scores_to_impute].isnull().sum())
print("\n--- New Data Types ---")
print(df_combined_data[['ReviewDate'] + numerical_scores_to_impute].dtypes)

--- After Imputation (Warnings Fixed) ---
ReviewDate                         0
EnvironmentSatisfaction            0
JobSatisfaction                    0
WorkLifeBalance                    0
RelationshipSatisfaction           0
TrainingOpportunitiesWithinYear    0
TrainingOpportunitiesTaken         0
SelfRating                         0
ManagerRating                      0
dtype: int64

--- New Data Types ---
ReviewDate                         datetime64[ns]
EnvironmentSatisfaction                     int64
JobSatisfaction                             int64
WorkLifeBalance                             int64
RelationshipSatisfaction                    int64
TrainingOpportunitiesWithinYear             int64
TrainingOpportunitiesTaken                  int64
SelfRating                                  int64
ManagerRating                               int64
dtype: object


In [ ]:
print(f"Current DataFrame shape: {df_combined_data.shape}")

Current DataFrame shape: (6899, 64)


In [ ]:
# Returns the total count of rows that are identical across ALL 64 columns
print(f"Total number of exact duplicate rows: {df_combined_data.duplicated().sum()}")

Total number of exact duplicate rows: 0


# **Feature Engineering: Calculating Employee Tenure**

In [ ]:
import pandas as pd
from datetime import datetime

# 1. Define a fixed reference date (e.g., the end of the study period)
reference_date = datetime(2022, 12, 31)

# 2. Convert HireDate to datetime format
df_combined_data['HireDate'] = pd.to_datetime(df_combined_data['HireDate'])

# 3. Calculate Tenure in Days
# (Reference Date - HireDate) / timedelta to get the total number of days as an integer
df_combined_data['Tenure_Days'] = (reference_date - df_combined_data['HireDate']).dt.days

# 4. Display the new feature
print("--- New Tenure Feature ---")
display(df_combined_data[['HireDate', 'Tenure_Days']].head())

--- New Tenure Feature ---


,HireDate,Tenure_Days
0,2012-01-03,4015
1,2012-01-03,4015
2,2012-01-03,4015
3,2012-01-03,4015
4,2012-01-03,4015


# **Final Data Split: Reporting vs. Modeling**

In [ ]:
# 1. Save the Descriptive DataFrame (includes text columns)
df_descriptive.to_csv('hr_data_descriptive.csv', index=False)

# 2. Save the Modeling DataFrame (purely numerical, optimized for ML)
df_model.to_csv('hr_data_modeling.csv', index=False)

print("✅ DataFrames saved successfully!")
print("Look for 'hr_data_descriptive.csv' and 'hr_data_modeling.csv' in your files explorer.")

✅ DataFrames saved successfully!
Look for 'hr_data_descriptive.csv' and 'hr_data_modeling.csv' in your files explorer.


In [ ]:
# 1. Redefine the columns to drop (including ethical filters)
columns_to_drop = [
    # Redundant Text/Date Columns (Existing drops)
    'EducationLevel',
    'JobSatisfaction_Text',
    'EnvironmentSatisfaction_Text',
    'SelfRating_Text',
    'ManagerRating_Text',
    'HireDate',
    'ReviewDate',
    # Ethical Filtering Columns (NEW drops)
    'Gender',
    'Age'
]

# 2. Re-create the Descriptive DataFrame (still includes all original columns)
df_descriptive = df_combined_data.copy()

# 3. Re-create the Modeling DataFrame (now drops 9 columns instead of 7)
df_model = df_combined_data.drop(columns=columns_to_drop, errors='ignore')

print("--- DataFrames Re-created Successfully ---")
print(f"1. df_descriptive (For Reporting): {df_descriptive.shape}") # Shape remains (6899, 65)
print(f"2. df_model (For Modeling): {df_model.shape}") # Shape is now (6899, 56)

--- DataFrames Re-created Successfully ---
1. df_descriptive (For Reporting): (6899, 35)
2. df_model (For Modeling): (6899, 26)


In [ ]:
# 1. Calculate the mean Job Satisfaction by Department
df_dept_satisfaction = df_combined_data.groupby('Department')[['JobSatisfaction']].mean().reset_index()
df_dept_satisfaction.rename(columns={'JobSatisfaction': 'Avg_Dept_JobSatisfaction'}, inplace=True)

# 2. Merge this new feature into the main modeling dataframes

# Merge into df_model
df_model = pd.merge(
    df_model,
    df_dept_satisfaction,
    on='Department',
    how='left'
)

# Merge into df_descriptive (for consistency/reporting)
df_descriptive = pd.merge(
    df_descriptive,
    df_dept_satisfaction,
    on='Department',
    how='left'
)

# Display the new feature
print("\n--- Aggregation Feature Added ---")
print(df_model[['Department', 'Avg_Dept_JobSatisfaction']].drop_duplicates().sort_values('Avg_Dept_JobSatisfaction'))
print(f"\nUpdated df_model shape: {df_model.shape}")


--- Aggregation Feature Added ---
         Department  Avg_Dept_JobSatisfaction
0             Sales                  3.422057
27       Technology                  3.434578
18  Human Resources                  3.435644

Updated df_model shape: (6899, 27)


In [ ]:
# 1. Define the columns to drop (redundant text and date objects)
columns_to_drop = [
    'EducationLevel',
    'JobSatisfaction_Text',
    'EnvironmentSatisfaction_Text',
    'SelfRating_Text',
    'ManagerRating_Text',
    'HireDate',
    'ReviewDate'
]

# 2. Create the Descriptive DataFrame (includes text columns)
# This is a copy of the current 64-column dataset.
df_descriptive = df_combined_data.copy()

# 3. Create the Modeling DataFrame (drops text columns)
# This creates the optimized 57-column set.
df_model = df_combined_data.drop(columns=columns_to_drop)


print("--- DataFrames Created Successfully ---")
print(f"1. df_descriptive (For Reporting): {df_descriptive.shape}")
print(f"2. df_model (For Modeling): {df_model.shape}")

--- DataFrames Created Successfully ---
1. df_descriptive (For Reporting): (6899, 35)
2. df_model (For Modeling): (6899, 28)


# **Final Data Preparation Summary**

This phase finalized the comprehensive HR dataset for downstream analysis. The key outcomes achieved in this data preparation process include:

Handling Missing Data: All missing performance, satisfaction, and rating scores were successfully imputed using the mode (Cell 70), a strategy appropriate for ordinal categorical data, and converted to an int64 format.

Feature Engineering: A crucial new feature, Tenure_Days, was calculated (Cell 74) to provide a high-resolution numerical measure of an employee's time with the company for use in predictive models.

Data Transformation: Key text-based categorical features (e.g., Gender, Department) were transformed into numerical format using One-Hot Encoding with drop_first=True (Cell 67) to eliminate multicollinearity and prepare the data for machine learning algorithms.

Final Output Split: The single, clean dataset was logically split into two distinct outputs for different use cases:

df_descriptive: Retains original text columns for clear business reporting and visualization.

df_model: An optimized, purely numerical dataset (58 columns) ready for direct consumption by predictive models (e.g., Logistic Regression or Tree-based models).